In [1]:
%load_ext autoreload
%autoreload 2
import networkx as nx
import matplotlib.pyplot as plt
import random
import json

#from pcc_implementacao import *
import pandas as pd
import plotly.express as px
from sklearn.neighbors import NearestNeighbors
import numpy as np

from model import *
import torch
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_networkx
from networkx.readwrite import json_graph
from collections import defaultdict
from model import PULearningPC, ParticleCompetitionModel
from aux import * 
from generate_dataset import *
from sklearn.metrics import f1_score


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# CORA DATASET

In [ ]:
cora_graph = load_cora_scar(
    k = 3, 
    positive_class_label=3, 
    percent_positive=0.1, 
    use_original_edges=True ,
    mst=False,
    #n_samples=
)

Loading Cora dataset...
Graph is not connected. Extracting the largest connected component.
Cora graph loaded: 2485 nodes, 5069 edges.


In [3]:
nx.is_connected(cora_graph)

True

In [4]:
cora_graph.number_of_nodes()

2485

In [44]:
max(cora_graph.nodes())

2707

In [ ]:
mcls_params = {'cluster_strategy': 'percentage',
               'positive_cluster_threshold': 0.1}
pcm_params = {
    'num_particles': 50,
    'average_node_potential_threshold': 0.8,
    'delta_p' : 0.8,
    'initialization_strategy': 'random'
}

model = PULearningPC(cora_graph,
                     num_neg = 200, 
                     pcm_params=pcm_params,
                     rns_params=mcls_params,
                     mlp_params=None)
model.train()
rn_cora = model.select_reliable_negatives()
rn_cora

--- Step 1: Running Label Initialization ---
Iteration: 1 | Avg Potential: 0.0500 | Coverage: 0.1557
Iteration: 2 | Avg Potential: 0.0500 | Coverage: 0.2439
Iteration: 3 | Avg Potential: 0.0619 | Coverage: 0.3034
Iteration: 4 | Avg Potential: 0.1002 | Coverage: 0.3360
Iteration: 5 | Avg Potential: 0.1368 | Coverage: 0.3654
Iteration: 6 | Avg Potential: 0.1705 | Coverage: 0.3819
Iteration: 7 | Avg Potential: 0.2058 | Coverage: 0.4056
Iteration: 8 | Avg Potential: 0.2382 | Coverage: 0.4193
Iteration: 9 | Avg Potential: 0.2685 | Coverage: 0.4423
Iteration: 10 | Avg Potential: 0.2983 | Coverage: 0.4628
Iteration: 11 | Avg Potential: 0.3280 | Coverage: 0.4837
Iteration: 12 | Avg Potential: 0.3472 | Coverage: 0.4970
Iteration: 13 | Avg Potential: 0.3652 | Coverage: 0.5103
Iteration: 14 | Avg Potential: 0.3863 | Coverage: 0.5247
Iteration: 15 | Avg Potential: 0.4019 | Coverage: 0.5384
Iteration: 16 | Avg Potential: 0.4178 | Coverage: 0.5509
Iteration: 17 | Avg Potential: 0.4283 | Coverage: 0.

Assigning cluster labels: 0it [00:00, ?it/s]
Finding best cluster by ratio: 0it [00:00, ?it/s]
Grouping nodes by cluster label: 100%|██████████| 2485/2485 [00:00<00:00, 5558850.90it/s]

Identified 0 reliable negatives.


[]

In [67]:
np.unique([cora_graph.nodes[n]['true_label'] for n in rn_cora])

array([0, 1])

In [52]:
y_true_f1 = [cora_graph.nodes[n]['true_label'] for n in rn_cora]
y_pred = [0] * len(rn_cora)
f1_score(y_true_f1, y_pred, pos_label = 0)

0.9473684210526315

# CITESEER

In [53]:
g_citeseer = load_citeseer_scar(
    positive_class_label=2,
    percent_positive=0.1,
    use_original_edges=True,
    mst=False
    )

Loading CiteSeer dataset...


Graph is not connected. Extracting the largest connected component.
CiteSeer graph loaded: 2120 nodes, 3679 edges.


Processing...
Done!


In [57]:
mcls_params = {'cluster_strategy': 'percentage',
               'positive_cluster_threshold': 0.1}
pcm_params = {
    'num_particles': 400,
    'average_node_potential_threshold': 0.8,
    'delta_p' : 0.8,
    'initialization_strategy': 'random'
}

model = PULearningPC(g_citeseer,
                     num_neg = 200, 
                     pcm_params=pcm_params,
                     mcls_params=mcls_params,
                     mlp_params=None)
model.train()
rn_citeseer = model.select_reliable_negatives()

--- Step 1: Running Label Initialization ---
Iteration: 1 | Avg Potential: 0.0500 | Coverage: 0.1887
Iteration: 2 | Avg Potential: 0.0500 | Coverage: 0.3212
Iteration: 3 | Avg Potential: 0.0770 | Coverage: 0.4198
Iteration: 4 | Avg Potential: 0.1388 | Coverage: 0.4741
Iteration: 5 | Avg Potential: 0.1908 | Coverage: 0.5066
Iteration: 6 | Avg Potential: 0.2381 | Coverage: 0.5330
Iteration: 7 | Avg Potential: 0.2792 | Coverage: 0.5467
Iteration: 8 | Avg Potential: 0.3118 | Coverage: 0.5590
Iteration: 9 | Avg Potential: 0.3494 | Coverage: 0.5759
Iteration: 10 | Avg Potential: 0.3780 | Coverage: 0.5910
Iteration: 11 | Avg Potential: 0.4037 | Coverage: 0.6009
Iteration: 12 | Avg Potential: 0.4240 | Coverage: 0.6094
Iteration: 13 | Avg Potential: 0.4394 | Coverage: 0.6208
Iteration: 14 | Avg Potential: 0.4509 | Coverage: 0.6255
Iteration: 15 | Avg Potential: 0.4589 | Coverage: 0.6325
Iteration: 16 | Avg Potential: 0.4703 | Coverage: 0.6349
Iteration: 17 | Avg Potential: 0.4799 | Coverage: 0.

Calculating mean dissimilarity: 100%|██████████| 1573/1573 [00:00<00:00, 14509.56it/s]

Identified 200 reliable negatives.


In [66]:
np.unique([g_citeseer.nodes[n]['true_label'] for n in rn_citeseer])

array([0, 1])

In [58]:
y_true_f1 = [g_citeseer.nodes[n]['true_label'] for n in rn_citeseer]
y_pred = [0] * len(rn_citeseer)
f1_score(y_true_f1, y_pred, pos_label = 0)

0.9743589743589743

# TWITCH

In [ ]:
twitch_g = load_twitch_scar(
    percent_positive=0.1,
mst=False, 
    )


Loading Twitch dataset from local files...
Sampling the first 16811 nodes from the largest component...
Twitch graph loaded: 16811 nodes, 66972 edges.


In [120]:
largest_cc = max(nx.connected_components(twitch_g), key=len)

In [121]:
twitch_g = twitch_g.subgraph(largest_cc).copy()

In [113]:
len(largest_cc)

13728

In [122]:
nx.is_connected(twitch_g)

True

In [123]:
np.unique([twitch_g.nodes[n]['true_label'] for n in twitch_g.nodes()], return_counts=True)

(array([0, 1]), array([6498, 7230]))

In [93]:
1912/6

318.6666666666667

In [95]:
mcls_params = {'cluster_strategy': 'percentage',
               'positive_cluster_threshold': 0.1}
pcm_params = {
    'num_particles': 318,
    'average_node_potential_threshold': 0.65,
    'delta_p' : 0.6,
    'movement_strategy' :'uniform',
    'initialization_strategy': 'random'
}

model = PULearningPC(twitch_g,
                     num_neg = 200, 
                     pcm_params=pcm_params,
                     mcls_params=mcls_params,
                     mlp_params=None)
model.train()
rn_twitch = model.select_reliable_negatives()

--- Step 1: Running Label Initialization ---
Iteration: 1 | Avg Potential: 0.0500 | Coverage: 0.1663
Iteration: 2 | Avg Potential: 0.0500 | Coverage: 0.2678
Iteration: 3 | Avg Potential: 0.0530 | Coverage: 0.3609
Iteration: 4 | Avg Potential: 0.0987 | Coverage: 0.4100
Iteration: 5 | Avg Potential: 0.1396 | Coverage: 0.4409
Iteration: 6 | Avg Potential: 0.1728 | Coverage: 0.4519
Iteration: 7 | Avg Potential: 0.2053 | Coverage: 0.4623
Iteration: 8 | Avg Potential: 0.2271 | Coverage: 0.4660
Iteration: 9 | Avg Potential: 0.2519 | Coverage: 0.4728
Iteration: 10 | Avg Potential: 0.2682 | Coverage: 0.4864
Iteration: 11 | Avg Potential: 0.2958 | Coverage: 0.5042
Iteration: 12 | Avg Potential: 0.3121 | Coverage: 0.5084
Iteration: 13 | Avg Potential: 0.3290 | Coverage: 0.5230
Iteration: 14 | Avg Potential: 0.3430 | Coverage: 0.5241
Iteration: 15 | Avg Potential: 0.3524 | Coverage: 0.5340
Iteration: 16 | Avg Potential: 0.3670 | Coverage: 0.5434
Iteration: 17 | Avg Potential: 0.3759 | Coverage: 0.

Calculating mean dissimilarity: 100%|██████████| 1308/1308 [00:00<00:00, 6940.72it/s]

Identified 200 reliable negatives.


In [98]:
np.unique([twitch_g.nodes[n]['true_label'] for n in twitch_g.nodes()], return_counts=True)

(array([0, 1]), array([1046,  866]))

In [99]:
y_true_f1 = [twitch_g.nodes[n]['true_label'] for n in rn_twitch]
y_pred = [0] * len(rn_twitch)
f1_score(y_true_f1, y_pred, pos_label = 0)

0.7460815047021944

# MNIST

In [3]:
g_mnist = load_mnist_scar(
    k=3,
    percent_positive=0.1,
    )

Loading MNIST dataset...
Building k-NN graph from features with k=3
Graph has 4 connected components. Connecting them using MST...
Graph is now fully connected with 70000 nodes and 164964 edges.
MNIST graph loaded: 70000 nodes, 164964 edges.


pegar uma amostra

In [3]:
nx.is_connected(g_mnist)

True

In [170]:
np.unique([g_mnist.nodes[n]['observed_label'] for n in g_mnist.nodes()], return_counts=True)

(array([0, 1]), array([66559,  3441]))

In [20]:
mcls_params = {'cluster_strategy': 'percentage',
               'positive_cluster_threshold': 0.2}
pcm_params = {
    'num_particles': 7000,
    'average_node_potential_threshold': 0.75,
    'coverage_graph_threshold': 0.8,
    'delta_p' : 0.6,
    'movement_strategy' :'degree_weighted',
    'initialization_strategy': 'degree_weighted'
}

model = PULearningPC(g_mnist,
                     num_neg = 200, 
                     pcm_params=pcm_params,
                     mcls_params=mcls_params,
                     mlp_params=None)
model.train()
rn_mnist = model.select_reliable_negatives()

--- Step 1: Running Label Initialization ---
Avg Potential: 0.0500 (Goal: 0.75), Coverage: 10.00% (Goal: 80%), Positive Cluster Found? False
Avg Potential: 0.0500 (Goal: 0.75), Coverage: 14.33% (Goal: 80%), Positive Cluster Found? False
Avg Potential: 0.0592 (Goal: 0.75), Coverage: 18.13% (Goal: 80%), Positive Cluster Found? True
Avg Potential: 0.0767 (Goal: 0.75), Coverage: 21.27% (Goal: 80%), Positive Cluster Found? True
Avg Potential: 0.0982 (Goal: 0.75), Coverage: 25.06% (Goal: 80%), Positive Cluster Found? True
Avg Potential: 0.1219 (Goal: 0.75), Coverage: 28.11% (Goal: 80%), Positive Cluster Found? True
Avg Potential: 0.1483 (Goal: 0.75), Coverage: 30.53% (Goal: 80%), Positive Cluster Found? True
Avg Potential: 0.1749 (Goal: 0.75), Coverage: 32.50% (Goal: 80%), Positive Cluster Found? True
Avg Potential: 0.2001 (Goal: 0.75), Coverage: 34.10% (Goal: 80%), Positive Cluster Found? True
Avg Potential: 0.2236 (Goal: 0.75), Coverage: 35.55% (Goal: 80%), Positive Cluster Found? True
Avg

Calculating mean dissimilarity: 100%|██████████| 61645/61645 [02:38<00:00, 388.46it/s]


Identified 200 reliable negatives.


In [23]:
np.unique([g_mnist.nodes[n]['true_label'] for n in rn_mnist], return_counts=True)

(array([0, 1]), array([ 77, 123]))

In [24]:
y_true_f1 = [g_mnist.nodes[n]['true_label'] for n in rn_mnist]
y_pred = [0] * len(rn_mnist)
f1_score(y_true_f1, y_pred, pos_label = 0)

0.555956678700361

# IONOSPHERE

In [13]:
g_ionosphere = load_ionosphere_scar(
    percent_positive=0.1,
    k = 3
    )

Loading Ionosphere dataset...
Building k-NN graph from features with k=3
Ionosphere graph loaded: 351 nodes, 896 edges.


In [3]:
nx.is_connected(g_ionosphere)

True

In [20]:
mcls_params = {'cluster_strategy': 'percentage',
               'positive_cluster_threshold': 0.3}
pcm_params = {
    'num_particles': 2,
    'average_node_potential_threshold': 0.8,
    'delta_p' : 0.6,
    'movement_strategy' :'uniform',
    'initialization_strategy': 'degree_weighted'
}

model = PULearningPC(g_ionosphere,
                     num_neg = 50, 
                     pcm_params=pcm_params,
                     mcls_params=mcls_params,
                     mlp_params=None)
model.train()
rn_ionosphere = model.select_reliable_negatives()

--- Step 1: Running Label Initialization ---
Iteration: 1 | Avg Potential: 0.0500 | Coverage: 0.0057
Iteration: 2 | Avg Potential: 0.0500 | Coverage: 0.0114
Iteration: 3 | Avg Potential: 0.0500 | Coverage: 0.0171
Iteration: 4 | Avg Potential: 0.0516 | Coverage: 0.0199
Iteration: 5 | Avg Potential: 0.0539 | Coverage: 0.0199
Iteration: 6 | Avg Potential: 0.0568 | Coverage: 0.0228
Iteration: 7 | Avg Potential: 0.0616 | Coverage: 0.0256
Iteration: 8 | Avg Potential: 0.0667 | Coverage: 0.0285
Iteration: 9 | Avg Potential: 0.0674 | Coverage: 0.0285
Iteration: 10 | Avg Potential: 0.0701 | Coverage: 0.0313
Iteration: 11 | Avg Potential: 0.0703 | Coverage: 0.0313
Iteration: 12 | Avg Potential: 0.0705 | Coverage: 0.0313
Iteration: 13 | Avg Potential: 0.0759 | Coverage: 0.0370
Iteration: 14 | Avg Potential: 0.0813 | Coverage: 0.0399
Iteration: 15 | Avg Potential: 0.0867 | Coverage: 0.0456
Iteration: 16 | Avg Potential: 0.0867 | Coverage: 0.0456
Iteration: 17 | Avg Potential: 0.0894 | Coverage: 0.

Calculating mean dissimilarity: 100%|██████████| 43/43 [00:00<00:00, 14054.01it/s]

Identified 43 reliable negatives.


In [21]:
y_true_f1 = [g_ionosphere.nodes[n]['true_label'] for n in rn_ionosphere]
y_pred = [0] * len(rn_ionosphere)
f1_score(y_true_f1, y_pred, pos_label = 0)

0.6129032258064516